In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [3]:
df = pd.read_csv('homework4data.csv')

In [4]:
for i in range(1, 11):
    df[f'R_dec{i}'] = df[f'R_dec{i}']/(1 + df['inflation'])

In [5]:
R = df.iloc[:,3:13].copy()
C = df.iloc[:, 13]

In [18]:
df

,year,quarter,inflation,R_dec1,R_dec2,R_dec3,R_dec4,R_dec5,R_dec6,R_dec7,R_dec8,R_dec9,R_dec10,c_growth
0,1951,1,0.036029,1.020913,1.044708,1.030644,1.001699,1.024529,1.016648,0.985931,0.979960,0.989741,0.994186,1.018434
1,1951,2,0.001932,1.020057,0.966831,0.977981,0.996064,1.094779,0.980945,0.988464,0.984889,0.975979,0.910829,0.991540
2,1951,3,0.003857,1.103889,1.127832,1.124744,1.113361,1.105242,1.123325,1.149766,1.147985,1.168448,1.167026,1.012247
3,1951,4,0.016904,0.994641,0.994440,0.989746,1.013669,1.001783,1.052100,1.000315,0.993793,0.980017,0.959764,1.002567
4,1952,1,-0.003022,1.018307,1.040207,1.035044,1.044681,1.026380,1.060304,1.075296,1.062240,1.016569,1.111785,0.997546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,2020,4,0.005281,1.118409,1.131008,1.115232,1.173002,1.162266,1.249483,1.180309,1.221172,1.329960,1.316982,1.008697
280,2021,1,0.013243,0.973560,1.039409,1.072095,1.073011,1.070868,1.153765,1.166956,1.172230,1.226833,1.266403,1.016547
281,2021,2,0.022364,1.087829,1.046829,1.073685,1.005746,1.025081,1.030578,1.018775,1.034164,1.051924,1.068275,1.028766
282,2021,3,0.012028,0.997909,0.979816,1.000960,0.959802,0.954546,0.962904,0.972831,0.980710,0.993707,1.017669,1.015357


## Orthogonal
By plugging $M_{t+1}$ in to $E_t(M_{t+1}R^R_{i,t+1}) = 1$, we have the orthogonality conditions:
$$
\mathbb{E} \left[ \beta \left( \frac{C_{t+1}}{C_t} \right)^{-\alpha} R_{i,t+1} - 1 \right] = 0
$$
for $i = 1, \ldots, 10,$

Define, $\theta = (\alpha, \beta)$
$$
f_t(\theta) = \begin{bmatrix}
    \beta \left( \frac{C_{t+1}}{C_t} \right)^{-\alpha} R_{1,t+1} - 1\\
    \vdots \\
    \beta \left( \frac{C_{t+1}}{C_t} \right)^{-\alpha} R_{10,t+1} - 1
\end{bmatrix}
$$

Then we have the othrogonal condition:
$$
g_T(\theta) = \frac{1}{T} \sum_{t=1}^T f_t(\theta)
$$

## implementation:
Clearly, $q = 2, r = 10, q < r$.
- First Stage:
$$
\hat{\theta}_1 = \arg \min_{\theta} [g_T(\theta)'] I [g_T(\theta)]
$$


- Second Stage:
$$
\hat{\theta}_2 = \arg \min_{\theta} [g_T(\theta)'] \hat{S}^{-1} [g_T(\theta)]
$$

Where S is the covariance matrix of f.
$$
\hat{S} = \frac{1}{T} \sum_{t=1}^T [f_t(\hat{\theta}_1)] [f_t(\hat{\theta}_1)]'
$$

## asymptotic variance:
$$
\text{asymp var}(\hat{\theta}_2) = \frac{1}{T} \left( \hat{D}' \hat{S}^{-1} \hat{D} \right)^{-1}
$$

where
$$
\hat{D} = \frac{\partial}{\partial \theta'} \frac{1}{T} \sum_{t=1}^T f_t(\hat{\theta}_2)
$$

In [6]:
R_np = np.array(R)
C_np = np.array(C)

In [7]:
R_np.shape,C_np.shape

((284, 10), (284,))

In [8]:
def othro(theta):
    b,a = theta

    # f = np.multiply(R_np, ((C_np)**(-a)).reshape(-1, 1)) 
    f = np.multiply(R_np, np.matrix(((C_np)**(-a))).T)
    f = b * f - 1
    return f

def obj(theta,W):
    b,a = theta
    f = othro(theta)
    g = f.mean(axis = 0)
    # W = np.identity(10)
    return (g@W@g.T)[0,0]

def getS(theta):
    T = R.shape[0]
    f = othro(theta)
    return (1/T)*f.T@f

def GMM1():
    theta_0 = [1,1]
    W = np.identity(10)
    model_1 = minimize(obj, x0=theta_0, args=(W))
    theta1 = model_1.x
    return theta1

def GMM2(theta_1):
    W = np.linalg.inv(getS(theta_1))
    model_2 = minimize(obj, x0=theta_1, args=(W))
    theta2 = model_2.x
    return theta2

#### full data set

In [9]:
theta1 = GMM1()
theta1

array([0.98028903, 1.00008814])

In [10]:
theta2 = GMM2(theta1)
theta2

array([ 0.97762048, -0.41463372])

#### data till 2019

In [11]:
df_ = df.set_index('year')
df_cut = df_.loc[:2019,:]
df_cut

,quarter,inflation,R_dec1,R_dec2,R_dec3,R_dec4,R_dec5,R_dec6,R_dec7,R_dec8,R_dec9,R_dec10,c_growth
year,,,,,,,,,,,,,
1951,1,0.036029,1.020913,1.044708,1.030644,1.001699,1.024529,1.016648,0.985931,0.979960,0.989741,0.994186,1.018434
1951,2,0.001932,1.020057,0.966831,0.977981,0.996064,1.094779,0.980945,0.988464,0.984889,0.975979,0.910829,0.991540
1951,3,0.003857,1.103889,1.127832,1.124744,1.113361,1.105242,1.123325,1.149766,1.147985,1.168448,1.167026,1.012247
1951,4,0.016904,0.994641,0.994440,0.989746,1.013669,1.001783,1.052100,1.000315,0.993793,0.980017,0.959764,1.002567
1952,1,-0.003022,1.018307,1.040207,1.035044,1.044681,1.026380,1.060304,1.075296,1.062240,1.016569,1.111785,0.997546
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,4,0.001237,0.842432,0.827606,0.890815,0.882957,0.855474,0.863667,0.843136,0.872271,0.818580,0.781356,1.002969
2019,1,0.006818,1.163104,1.152235,1.110635,1.124476,1.136587,1.137948,1.093958,1.061177,1.115110,1.089859,1.000840
2019,2,0.004279,1.038242,1.047690,1.023369,1.036786,1.046448,1.038050,0.987663,1.046942,1.041554,1.048718,1.005626


In [12]:
R_ = df_cut.iloc[:,2:12].copy()
C_ = df_cut.iloc[:, 12]
R_np_c = np.array(R_)
C_np_c = np.array(C_)

In [15]:
def othro(theta):
    b,a = theta

    # f = np.multiply(R_np, ((C_np)**(-a)).reshape(-1, 1)) 
    f = np.multiply(R_np_c, np.matrix(((C_np_c)**(-a))).T)
    f = b * f - 1
    return f

def obj(theta,W):
    b,a = theta
    f = othro(theta)
    g = f.mean(axis = 0)
    # W = np.identity(10)
    return (g@W@g.T)[0,0]

def getS(theta):
    T = R.shape[0]
    f = othro(theta)
    return (1/T)*f.T@f

def GMM1():
    theta_0 = [1,1]
    W = np.identity(10)
    model_1 = minimize(obj, x0=theta_0, args=(W))
    theta1 = model_1.x
    return theta1

def GMM2(theta_1):
    W = np.linalg.inv(getS(theta_1))
    model_2 = minimize(obj, x0=theta_1, args=(W))
    theta2 = model_2.x
    return theta2

In [16]:
theta1_c = GMM1()
theta1_c

array([0.98085905, 1.00008918])

In [17]:
theta2_c = GMM2(theta1_c)
theta2_c

array([ 1.18192943, 46.05749789])

By compare the result of the full dataset, the outliers from 2020-2021 bias our estimation of $\alpha$ to a much smaller value.

## Asymptotic covariance matrix

$$
\sqrt{T} (\hat{b}_T - b_0) \overset{L}{\rightarrow} N(0,V)
$$
$$
V = (D'_0 W D_0)^{-1} (D'_0 W S W D_0) (D'_0 W D_0)^{-1} = (D'_0 W D_0)^{-1}
$$

$$
AsymV = V/T
$$
$$
D_0 = E \left[ \frac{\partial f_t(b_0)}{\partial b'} \right] =\begin{pmatrix} \dfrac{\partial f_t(\alpha, \beta)}{\partial \alpha} \\ \dfrac{\partial f_t(\alpha, \beta)}{\partial \beta} \end{pmatrix}^{'} = \begin{pmatrix} \beta R_{1, t+1}^R (\dfrac{C_{t+1}}{C_t})^{-\alpha}ln(\dfrac{C_{t+1}}{C_t}) \\ (\dfrac{C_{t+1}}{C_t})^{-\alpha} R_{1, t+1}^R \end{pmatrix}^{'}
$$

#### 2021 data

In [38]:
a = theta2[1]
b = theta2[0]


# Reshape C_np to perform element-wise operations with R_np
C_np_reshaped = C_np.reshape(-1, 1)

# Compute the common terms
C_power_neg_a = C_np_reshaped ** (-a)  # shape (284, 1)
R_times_C_power_neg_a = R_np * C_power_neg_a  # shape (284, 10)
ln_C = np.log(C_np_reshaped)  # shape (284, 1)

# Compute the Jacobian terms
jacobian_first_term = b * R_times_C_power_neg_a * ln_C  # shape (284, 10)
jacobian_second_term = C_power_neg_a * R_np  # shape (284, 10)

# Combine the terms to form the full Jacobian for each R_ij
# Stacking the Jacobian matrices horizontally
# Note: We repeat ln_C and C_power_neg_a across the second dimension (axis=1) to match the shape of R_np
Jacobian = np.stack((jacobian_first_term, jacobian_second_term), axis=2)  # shape (284, 10, 2)

D_0 = Jacobian.mean(axis=0)

In [39]:
W_full = getS(theta2)

asym_var = np.linalg.inv(np.matmul(np.matmul(np.transpose(D_0), W_full), D_0))/ (len(df))
asym_var

matrix([[ 2.15077229e+08, -9.71260198e+05],
        [-9.71260198e+05,  4.38608740e+03]])

In [40]:
SE = np.diag((asym_var))**(1/2)
SE

array([14665.5115348 ,    66.22754257])

#### 2019 data

In [41]:
a = theta2_c[1]
b = theta2_c[0]


# Reshape C_np to perform element-wise operations with R_np
C_np_reshaped = C_np_c.reshape(-1, 1)

# Compute the common terms
C_power_neg_a = C_np_reshaped ** (-a)  # shape (284, 1)
R_times_C_power_neg_a = R_np_c * C_power_neg_a  # shape (284, 10)
ln_C = np.log(C_np_reshaped)  # shape (284, 1)

# Compute the Jacobian terms
jacobian_first_term = b * R_times_C_power_neg_a * ln_C  # shape (284, 10)
jacobian_second_term = C_power_neg_a * R_np_c  # shape (284, 10)

# Combine the terms to form the full Jacobian for each R_ij
# Stacking the Jacobian matrices horizontally
# Note: We repeat ln_C and C_power_neg_a across the second dimension (axis=1) to match the shape of R_np
Jacobian = np.stack((jacobian_first_term, jacobian_second_term), axis=2)  # shape (284, 10, 2)

D_0_ = Jacobian.mean(axis=0)

W_full_ = getS(theta2_c)

asym_var_ = np.linalg.inv(np.matmul(np.matmul(np.transpose(D_0_), W_full_), D_0_))/ (len(df))
asym_var_

matrix([[ 2.90732010e+08, -1.23322363e+06],
        [-1.23322363e+06,  5.23107440e+03]])

In [42]:
SE_ = np.diag((asym_var_))**(1/2)
SE_

array([17050.86537737,    72.3261668 ])

## HJ-test:
$$
J = T \cdot [g_T(\hat{\theta}_2)'] \hat{S}^{-1} [g_T(\hat{\theta}_2)], \quad J \overset{d}{\rightarrow} \chi^2(r - q) = \chi^2(8)
$$


#### 2021 data

In [52]:
def othro(theta):
    b,a = theta

    # f = np.multiply(R_np, ((C_np)**(-a)).reshape(-1, 1)) 
    f = np.multiply(R_np, np.matrix(((C_np)**(-a))).T)
    f = b * f - 1
    return f

def obj(theta,W):
    b,a = theta
    f = othro(theta)
    g = f.mean(axis = 0)
    # W = np.identity(10)
    return (g@W@g.T)[0,0]

In [53]:
from scipy.stats import chi2
W_full = np.linalg.inv(getS(theta2))
J = obj(theta2,W_full) * len(df)
p_val = 1 - chi2.cdf(J, df= 8)

J, p_val

(17.60493319835785, 0.024391436474068495)

#### 2019 data

In [54]:
def othro(theta):
    b,a = theta

    # f = np.multiply(R_np, ((C_np)**(-a)).reshape(-1, 1)) 
    f = np.multiply(R_np_c, np.matrix(((C_np_c)**(-a))).T)
    f = b * f - 1
    return f

def obj(theta,W):
    b,a = theta
    f = othro(theta)
    g = f.mean(axis = 0)
    # W = np.identity(10)
    return (g@W@g.T)[0,0]

In [55]:
W_full_ = np.linalg.inv(getS(theta2_c))
J = obj(theta2_c,W_full_) * len(df)
p_val = 1 - chi2.cdf(J, df= 8)

J, p_val

(10.959951469679448, 0.20397874714106146)

We can't reject the null hypothesis if we exclude the covid data. The covid outlier make the model estimation not statistically significant.

## Q2